In [1]:
import os
import re
import pandas as pd

# E coli

In [2]:
# -- dropping out all genome ids that we don't have a corresponding entry for in genome_ids file

species='Escherichia_coli'; drug=''

presence_df = pd.read_csv(f'../data/presence_matrices/{species}_GxS.csv', index_col=0); 
X_df=presence_df.T
X_df.index = X_df.index.astype('float')

genome_ids_file = f'../pangenome-repo/Pangenome-Analysis-Workflow/genome_ids/{species}_genome_ids.csv'
samples_df = pd.read_csv(genome_ids_file, dtype=str)
samples_list=samples_df['genome.genome_ids'].tolist()

In [12]:
#  remove all genome ids that are not in the genome_ids file

for file in os.listdir('../metadata/Escherichia_coli/'):
    if file.endswith('.csv'):
        y=pd.read_csv(f'../metadata/Escherichia_coli/{file}', dtype=str)
        old=y.shape[0]
        # print(y)
        y=y[y['genome_id'].isin(samples_list)]
        new=y.shape[0]
        print(f'{file} - {old-new} samples removed')
        y.to_csv(f'../metadata/Escherichia_coli/{file}', index=False)

In [23]:
# -- plotting all pheno data for all available drugs

df = pd.DataFrame(columns=['drug', 'R', 'S'])

for file in os.listdir('../metadata/Escherichia_coli/'):
    if file.endswith('.csv'):
        drug = re.match(r'Escherichia_coli_(.*).csv', file).group(1)
        data = pd.read_csv(f'../metadata/Escherichia_coli/{file}')
        # count the number of 1s in SIR col and save in a count var
        count = data['SIR'].value_counts()
        count_R=count.get(1, 0)
        count_S=count.get(0, 0)
        # print(f'{drug}: {count}')
        # print('count of 1s:', count_R)
        # print('count of 0s:', count_S)
        if count_R == 0 and count_S == 0:
            continue
        df = df.append({'drug': drug, 'R': count_R, 'S': count_S}, ignore_index=True)
df.set_index('drug', inplace=True)

# -- adding percentage columns
df['R%'] = df['R'] / (df['R'] + df['S']) * 100
df['S%'] = df['S'] / (df['R'] + df['S']) * 100  
df['samples no.'] = df['R'] + df['S']
# df

/tmp/ipykernel_25912/1609644438.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'drug': drug, 'R': count_R, 'S': count_S}, ignore_index=True)
/tmp/ipykernel_25912/1609644438.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'drug': drug, 'R': count_R, 'S': count_S}, ignore_index=True)
/tmp/ipykernel_25912/1609644438.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'drug': drug, 'R': count_R, 'S': count_S}, ignore_index=True)
/tmp/ipykernel_25912/1609644438.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'drug': drug, 'R': count_R, 'S': count_S}, ignore_inde

In [24]:
#  -- filtering out those drugs with over a 70/30 class imbalance and less than 50 samples
df = df[(df['R%'] < 70) & (df['S%'] < 70) & (df['samples no.'] > 50)]
df.to_csv(f'../metadata/{species}_pheno_data.csv')

In [25]:
df

,R,S,R%,S%,samples no.
drug,,,,,
streptomycin,55,78,41.353383,58.646617,133
sulfamethoxazole,83,50,62.406015,37.593985,133
tetracycline,140,85,62.222222,37.777778,225
cefalothin,196,170,53.551913,46.448087,366
trimethoprim_sulphamethoxazole,180,96,65.217391,34.782609,276
trimethoprim,216,283,43.286573,56.713427,499
amoxicillin,596,388,60.569106,39.430894,984
ampicillin,312,194,61.660079,38.339921,506
doripenem,27,52,34.177215,65.822785,79


In [27]:
import plotly.graph_objects as go

trace1 = go.Bar(x=df.index, y=df[df.columns[0]], name=df.columns[0])
trace2 = go.Bar(x=df.index, y=df[df.columns[1]], name=df.columns[1])

fig = go.Figure(data=[trace1, trace2])

fig.update_layout(barmode='stack', width=800, height=600)
fig.show()

In [3]:
os.getcwd()

'/home/rayane/capstone-project/src'